In [4]:
import segmentation_models_pytorch as smp

# scSE Unet-vgg
unet_vgg = smp.Unet(
    encoder_name="vgg11",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    # encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    decoder_attention_type ='scse',
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)

# scSE Unet
unet_scse = smp.Unet(
    encoder_name='efficientnet-b5',
#     encoder_weights='imagenet', # use `imagenet` pre-trained weights for encoder initialization
#     decoder_use_batchnorm=True,
    decoder_attention_type='scse',
    in_channels=1, # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3, # model output channels (number of classes in your dataset)
)

In [1]:
import torch
import torch.quantization

class E4M3FakeQuant(torch.quantization.FakeQuantize):
    def __init__(self):
        super().__init__(
            observer=torch.quantization.MinMaxObserver,
            quant_min=-8,  # Adjust based on e4m3 range
            quant_max=7,
            dtype=torch.qint8,  # Closest available
        )

# Apply to a model
model = torch.nn.Linear(10, 10)
model.weight_fake_quant = E4M3FakeQuant()